# Bayesian Hyperperameter Optimization

## Objective
This is an exercise in learning how to implement Baysian optimization for tuning the hyperparameters of a machine learning model.

I tried to keep everything as simple as possible. Data engineering was kept to a minimum, and the basic model structure was very straightforward.

## Results
This notebook successfully achieves the following:
1. Defines a generating function to construct a model with variable hyperparameters
2. Defines an evaluation function that scores a model with variable hyperparameters
3. Applies Bayesian optimization to probe a parameter space for potentially good hyperparameter values
4. Allows for easy manipulation of several assumptionsm bounds, and settings for the optimization task
5. Stores the progress of the optimizer so for reference or reuse

# References and Data

## References

This code that follows was heavily inspired by/ taken from code published by Jeff Heaten for his course *T81 558:Applications of Deep Neural Networks*. It can be found on [github](https://github.com/jeffheaton/t81_558_deep_learning/tree/master)

Here is the citation for that course:
```
cff-version: 1.2.0
message: "If you use this software, please cite it as below."
authors:
- family-names: "Jeff"
  given-names: "Heaton"
  orcid: "https://orcid.org/0000-0003-1496-4049"
title: "Applications of Deep Neural Networks"
version: 2021.08.01
date-released: 2021-08-01
url: "https://arxiv.org/abs/2009.05673"
```

## Data

The dataset used to for demonstration purposes is the "California Housing" dataset found [here](https://www.kaggle.com/datasets/camnugent/california-housing-prices)

# Loading Packages and Helper Functions

There are a bunch of unused packages here, but my goal when making this notebook was to build a basic template that would be easy to adapt to future tasks.

The `hms_string` function used to make runtime benchmark outputs more readable.

In [1]:
#!pip install bayesian-optimization

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
import io
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import numpy as np
import pandas as pd
import statistics
import requests
from scipy.stats import zscore
from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import ShuffleSplit
import tensorflow as tf
import tensorflow.keras.initializers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer
from tensorflow.keras.layers import LeakyReLU,PReLU
from tensorflow.keras.optimizers.legacy import Adam # Note: legacy is for M1/M2 issues
import time



# Nicely format a time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# Preparing the Data

For this dataset, I did the following preparation:
1. Load the dataset from a csv file to a `pandas` dataframe
2. Dropped rows with missing data (this only eliminates 270 of 20,640 rows)
3. Identified a target variable: `median_house_value`
4. Replaced the categorical feature, `ocean_proximity`, with dummies
5. Normalized the non-categrical features using z-scores
6. Stored the features and target as `numpy` arrays

In [2]:
# Loading the data
df = pd.read_csv('housing.csv')

In [3]:
# Drop any rows with NA values
df.dropna(inplace=True)

In [4]:
# Keep track of the target column (or columns)
y_column = 'median_house_value'

# Keep track of any categorical features to make dummies for
categorical_features = ['ocean_proximity']

# Normalize the values in non-target and non-categorical columns
for feature in df.drop(y_column, axis=1).drop(categorical_features, axis=1).columns:
    df[feature]=zscore(df[feature])

# Create dummies for all of the categorical features
# and drop the original feature column
for feature in categorical_features:
    dummies = pd.get_dummies(df[feature], drop_first=True, prefix=feature)
    df = pd.concat([df,dummies],axis=1)
    df.drop(feature, axis=1, inplace=True)

# Keep track of the feature columns that will be used in the model
x_columns = df.drop(y_column, axis=1).columns

In [5]:
####################
# For regression or binary classfication:
x = df[x_columns].values
y = df[y_column].values
####################
# For multiclassification:
#x = df[x_columns].values
#dummies = pd.get_dummies(df[y_column])
#y_columns = dummies.columns
#y = dummies.values
####################

# Hyperparameter Optimization

## Tools

I want to use Bayesian optimization for tuning the parameters of my model. I will be using the `bays_opt` package for this task ([source](https://github.com/bayesian-optimization/BayesianOptimization)).

At the very least, the optimizer needs two things:
1. A "black box" funtion that provides an evaluation, or score, for a model with given parameter values
2. The domain from which parameter values may be selected

## Implementation and Settings

The steps in this implementaion are:
1. Write a function that will construct a model given a set of parameter values:
    - `dropout_rate` - the percent of neurons to drop out between hidden layers
    - `neuronPct` - used to set the number of neurons in the first hidden layer as `neuronPct * MAX_NEURONS`
    - `neuronShrink` - used to control how quicky the number of neurons in each successive hidden layers drops as a percent 
        - e.g. if `neuronShrink=.7` then the number of neurons in each hidden layer will be 30% fewer than the previous layer
2. Write a function that evaluates a model (gives a score) given a set of parameter values:
    - `dropout_rate`
    - `neuronPct`
    - `neuronShrink` 
    - `learning_rate` - the learning rate for the `Adam` optimizer when compiling the model

In this implementation, the Bayesian optimizer will vary these optimization parameters (`dropout_rate`, `neuronPct`, `neuronShrink`, `learning_rate`) to try and maximize the score for the model.

We can set ranges for each of these parameters as a dictionary, e.g.:

```
pbounds = {'dropout_rate': (0.0, 0.499),
           'learning_rate': (0.0, 0.1),
           'neuronPct': (0.01, 1),
           'neuronShrink': (0.01, 1)
          }
```

Some important choices and settings determined manually, outside of these automatically tuned hyperparameters:
- Constucting the model:
    - `MAX_NEURONS` - the largest number of neurons possible in the first hidden layer
    - `MIN_NEURONS` - the least number of neurons allowed in a hidden layer before no additional layers are added (the last hidden layer could have `neuronPct*(MIN_NEURONS+1)` neurons)
    - `MAX_LAYERS` -  the most hidden layers that will be added to the model
    - The activation function in each hidden layer, here: `PReLU()`
    - The regularization strategy implemented in each hidden layer, here: `None`
    - The activation function of the output layer, here: `linear`
- Training and evaluating:
    - Resampling strategy: *bootstrapping*
        - `SPLITS` - the number of bootstrapping itterations to perform for each 
        - `TEST_SIZE` - the portion of the data to use as the test set in each bootstrap itteration
     - Model fitting:
        - `EPOCHS` - the maximum number of epochs to perform for each
        - Early stopping
            - `PATIENCE` - the number of epochs to complete without seeing improvement
- Bayesian Optimization:
    - `INIT_POINTS` - the number of random parameter values to probe before attempting to optimize
    - `N_ITER` - the number of iterations to perform for optimization

In [11]:
# Model bounds:
MAX_NEURONS = 5000
MIN_NEURONS = 25
MAX_LAYERS = 10

# Bootstrapping settings:
SPLITS = 4
TEST_SIZE = 0.2
print_bootstrap_stats = True

# Model fitting and early stopping settings:
EPOCHS = 18
PATIENCE = 9

# Bayesian optimizer settings
INIT_POINTS = 25
N_ITER = 50
OPTIMIZER_RANDOM_STATE = 33

In [7]:
# Model generating function
def generate_model(dropout_rate, neuronPct, neuronShrink):
    # We start with some percent of 5000 starting neurons on 
    # the first hidden layer.
    neuronCount = int(neuronPct * MAX_NEURONS)
    
    # Construct neural network
    model = Sequential()

    # So long as there would have been at least MIN_NEURONS neurons and 
    # fewer than MAX_LAYERS layers, create a new layer.
    layer = 0
    while neuronCount>MIN_NEURONS and layer<MAX_LAYERS:
        # The first (0th) layer needs an input input_dim(neuronCount)
        if layer==0:
            model.add(Dense(neuronCount, 
                input_dim=x.shape[1], 
                activation=PReLU()))
        else:
            model.add(Dense(neuronCount, activation=PReLU())) 
        layer += 1

        # Add dropout after each hidden layer except the last one
        if layer<MAX_LAYERS-1:
            model.add(Dropout(dropout_rate))

        # Shrink neuron count for each layer
        neuronCount = neuronCount * neuronShrink

    # Add the output layer
    ####################
    # For regression:
    model.add(Dense(1))
    ####################
    # For binary classification:
    # model.add(Dense(y.shape[1],activation='sigmoid')) # Output
    ####################
    # For multiclassification:
    # model.add(Dense(y.shape[1],activation='softmax')) # Output
    ####################
    return model  

In [8]:
# Model evaluating function
def evaluate_network(dropout_rate, neuronPct, neuronShrink, learning_rate):
    # Set up bootstrapping
    
    # For Regression:
    boot = ShuffleSplit(n_splits=SPLITS, test_size=TEST_SIZE)
    # For Classification:
    #boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1)
    
    # Track progress
    mean_benchmark = []
    epochs_needed = []
    num = 0
    
    # Loop through samples
    ####################
    #For regression:
    for train, test in boot.split(x,y):
    ####################
    #For classification:
    #for train, test in boot.split(x,df[y_column]):
    ####################
        start_time = time.time()
        num+=1

        # Split train and test
        x_train = x[train]
        y_train = y[train]
        x_test = x[test]
        y_test = y[test]

        # Generate the model using the function we defined
        model = generate_model(dropout_rate, neuronPct, neuronShrink)
        
        # Compile the model
        
        ####################
        #For regression:
        model.compile(loss='mean_squared_error',
                      optimizer=Adam(learning_rate=learning_rate))
        ####################
        #For classification:
        #model.compile(loss='categorical_crossentropy', 
        #              optimizer=Adam(learning_rate=learning_rate))
        ####################
        monitor = EarlyStopping(monitor='val_loss',
                                min_delta=1e-3, 
                                patience=PATIENCE,
                                verbose=0, 
                                mode='auto', 
                                restore_best_weights=True)

        # Train on the bootstrap sample
        model.fit(x_train,
                  y_train,
                  validation_data=(x_test,y_test),
                  callbacks=[monitor],
                  verbose=0,
                  epochs=EPOCHS)
        
        # Keep track of needed epochs
        epochs_used = monitor.stopped_epoch if monitor.stopped_epoch > 0 else EPOCHS
        epochs_needed.append(epochs_used)

        # Predict on the out of boot (validation)
        pred = model.predict(x_test, 
                             verbose=0)

        # Measure this bootstrap's log loss
        ####################
        # For regression:
        score = np.sqrt(metrics.mean_squared_error(pred,y_test))
        ####################
        # For classification:
        #y_compare = np.argmax(y_test,axis=1) # For log loss calculation
        #score = metrics.log_loss(y_compare, pred)
        ####################
        
        mean_benchmark.append(score)
        m1 = statistics.mean(mean_benchmark)
        m2 = statistics.mean(epochs_needed)
        mdev = statistics.pstdev(mean_benchmark)
        
        # Print stats for this iteration
        if print_bootstrap_stats:
            time_took = time.time() - start_time
        
            print(f"#{num}: score (rmse)={score:.6f}, mean score={m1:.6f},",
                  f" stdev={mdev:.6f}",
                  f" epochs={epochs_used}, mean epochs={int(m2)}",
                  f" time={hms_string(time_took)}")
        
    tensorflow.keras.backend.clear_session()
    
    return (-m1)

In [ ]:
# Running Bayesian optimiztion

# Supress NaN warnings
import warnings
warnings.filterwarnings("ignore",category = RuntimeWarning)

# Bounded region of parameter space
pbounds = {'dropout_rate': (0.0, 0.499),
           'learning_rate': (0.0, 0.1),
           'neuronPct': (0.01, 1),
           'neuronShrink': (0.01, 1)
          }

optimizer = BayesianOptimization(
    f=evaluate_network,
    pbounds=pbounds,
    verbose=2,  
    # verbose = 1 prints only when a maximum is observed
    # verbose = 0 is silent
    random_state=OPTIMIZER_RANDOM_STATE,
)

# Store logs of the probes for later use
# See docs for load_log tools
logger = JSONLogger(path='logs.log',
                    reset=True # False: don't clear existing log
                   )
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

start_time = time.time()
optimizer.maximize(init_points=INIT_POINTS, n_iter=N_ITER,)
time_took = time.time() - start_time

print(f"Total runtime: {hms_string(time_took)}")
print(optimizer.max)